## Cosine

In [5]:
# there are bugs - overflow somewhere!!!

import cv2
import numpy as np

# cho trước chiều rộng và chiều cao của ảnh
height = 288
width  = 384

def compute_integral_image(data):
    sum_axis_0 = np.cumsum(data, axis=0)
    sum_axis_1 = np.cumsum(sum_axis_0, axis=1)
    return sum_axis_1

def stereo_matching(left_img, right_img, disparity_range):   
    # đọc ảnh trái và ảnh phải, rồi chuyển sang ảnh grayscale
    left  = cv2.imread(left_img, 0)    
    right = cv2.imread(right_img, 0)  
    
    left  = left.astype(np.float32)
    right = right.astype(np.float32)
    
    # compute integral image space
    integral_xy = np.full((height, width, disparity_range), 255*255, np.float32)
    integral_xx = np.full((height, width), 255*255, np.float32)
    integral_yy = np.full((height, width), 255*255, np.float32)    
   
    integral_xx = compute_integral_image(left*left)
    integral_yy = compute_integral_image(right*right)
    for j in range(disparity_range):        
        integral_xy[:, j:width, j] = left[:,j:width]*right[:, :width-j]
        integral_xy[:,:,j] = compute_integral_image(integral_xy[:,:,j])
        
    # compute sum
    side = 2
    integral_xy_sum = np.full((height, width, disparity_range), 255*255, np.float32)    
    integral_xy_sum[:height-side, :width-side, :] = integral_xy[side:, side:, :]
    integral_xy_sum[side+1:, side+1:, :] = integral_xy_sum[side+1:, side+1:, :] + integral_xy[:height-side-1, :width-side-1, :]
    integral_xy_sum[side+1:, :width-side, :] = integral_xy_sum[side+1:, :width-side, :] - integral_xy[:height-side-1, side:, :]
    integral_xy_sum[:height-side, side+1:, :] = integral_xy_sum[:height-side, side+1:, :] - integral_xy[side:, :width-side-1, :]
    
    integral_xx_sum = np.full((height, width), 255*255, np.float32)    
    integral_xx_sum[:height-side, :width-side] = integral_xx[side:, side:]
    integral_xx_sum[side+1:, side+1:] = integral_xx_sum[side+1:, side+1:] + integral_xx[:height-side-1, :width-side-1]
    integral_xx_sum[side+1:, :width-side] = integral_xx_sum[side+1:, :width-side] - integral_xx[:height-side-1, side:]
    integral_xx_sum[:height-side, side+1:] = integral_xx_sum[:height-side, side+1:] - integral_xx[side:, :width-side-1]
    
    integral_yy_sum = np.full((height, width), 255*255, np.float32)    
    integral_yy_sum[:height-side, :width-side] = integral_yy[side:, side:]
    integral_yy_sum[side+1:, side+1:] = integral_yy_sum[side+1:, side+1:] + integral_yy[:height-side-1, :width-side-1]
    integral_yy_sum[side+1:, :width-side] = integral_yy_sum[side+1:, :width-side] - integral_yy[:height-side-1, side:]
    integral_yy_sum[:height-side, side+1:] = integral_yy_sum[:height-side, side+1:] - integral_yy[side:, :width-side-1]
    
    # compute cost
    cost_space = np.full((height, width, disparity_range), -1.0, np.float32)
    for j in range(disparity_range):
        denor = np.sqrt(integral_xx_sum[:, j:width])*np.sqrt(integral_yy_sum[:, :width-j])
        cost_space[:, j:width, j] = integral_xy_sum[:, j:width, j] / denor
        
    # compute disparity from cost space
    scale = 255 / disparity_range    
    depth = np.argmax(cost_space, axis=2)
    depth = depth*scale
    depth = depth.astype(np.uint8)
                
    # chuyển dữ liệu từ ndarray sang kiểu Image và lưu xuống file
    cv2.imwrite('images/disparity_cosine.png', depth)  
         
if __name__ == '__main__':
    disparity_range = 16 # cho cặp ảnh Tsukuba
    stereo_matching("images/left.png", "images/right.png", disparity_range)

C:\Users\Dinh Vinh\AppData\Local\Temp\ipykernel_12616\2113956217.py:58: RuntimeWarning: invalid value encountered in sqrt
  denor = np.sqrt(integral_xx_sum[:, j:width])*np.sqrt(integral_yy_sum[:, :width-j])
